<a href="https://colab.research.google.com/github/az3ertyuiop/236587497874124878547852147557-/blob/gc/comparaison_chr5_10%2C000_200%2C000vertebres.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


On cherche à curl les différents types d'ARN présents dans la region 10000:200 000 du chromosome 5 des vertébrés afin d'en faire une analyse via python (analyse par composante principale ou régression linéraire/stats).

In [ ]:
%%shell

curl -s "https://rnacentral.org/api/v1/overlap/region/homo_sapiens/5:10,000-200,000" | sed -E 's/[{.*,}]/\n/g' | egrep "biotype" | cut -d ":" -f 2 | sort | uniq | sed -E 's/^"(.*[^"])"$/\1/g'
#la cmd ci-dessus permet d'obtenir les différents types d'ARN transcrits dans la région 10,000-200,000 du chromosome 5 d'H.Sapiens.


lncRNA
piRNA
sRNA
Y_RNA


Cette commande va nous servir de base afin d'automatiser l'obtention des différents types d'ARN de cette région selon différents organismes


In [ ]:
%%shell
curl -s https://rnacentral.org/help/genomic-mapping#species-with-genomic-mapping | egrep "?species=" | tr -d " " | cut -d "=" -f 3 | sed -E 's/([a-z_]*).*/\1/g' >> /content/drive/MyDrive/liste_sp_mappees-rnacentral.txt


#on récupère tout les organismes dont le génome est mappé sur rnacentral.
#on peut ainsi en faire une liste et utiliser cette liste dans la première commande à l'aide de "xargs"

In [ ]:
%%shell

more /content/drive/MyDrive/liste_sp_mappees-rnacentral.txt | xargs -I % curl -s "https://rnacentral.org/api/v1/overlap/region/%/5:10,000-200,000" | sed -E 's/[{.*,}]/\n/g' | egrep "biotype" | cut -d ":" -f 2 | sort | uniq | sed -E 's/^"(.*[^"])"$/\1/g'


In [ ]:
%%shell

curl -s "https://rnacentral.org/api/v1/overlap/region/nom_sp/5:10,000-200,000" | sed -E 's/[{.*,}]/\n/g' | egrep "biotype" | cut -d ":" -f 2 | sort | uniq | sed -E 's/^"(.*[^"])"$/\1/g'

piRNA
tRNA


Pour le déroulement de la suite, j'ai besoin de trier les espèces mappées sur rnacentral afin d'en sortir uniquement les espèces mappées au chr5 dans la région souhaitée. Ainsi je vais devoir combiner python et bash.

In [ ]:
import subprocess

#Script pour déterminer si l'espèce a la région souhaitée ou non
index='/content/drive/MyDrive/liste_sp_mappees-rnacentral.txt'
with open(index, 'r') as x:
    L = [line.strip() for line in x]


#trions maintenant selon l'output du chr5
#pour les tests essayons avec L2
L2=['aspergillus_niger','homo_sapiens']

for nom_sp in L[:]:
  bash=f"curl -s https://rnacentral.org/api/v1/overlap/region/{nom_sp}/5:10000-200000 | sed -E 's/[{{.*,}}]/\\n/g' | egrep 'biotype' | cut -d ':' -f 2 | sort | uniq | sed -E 's/^\"(.*[^\"])\"$/\\1/g'"
  result = subprocess.run(bash, shell=True, capture_output=True, text=True)
  print(result)
  if not result.stdout.strip(): #on trie selon le "stdout" qui est soit vide -> stdout='' soit rempli stdout='lncRNA\npiRNA\nsRNA\nY_RNA\n'
    L.remove(nom_sp)

#À la fin de ce script, L contient uniquement les espèces mappées.
print(L)

index2='/content/drive/MyDrive/liste_sp_mappees.txt'

#On enregistre la list des sp mappees en .txt.
with open(index2, 'w') as liste:
    for item in L:
        liste.write(f"{item}\n")


J'ai maintenant ma liste des espèces mappées sur la région souhaitée. Je peux extraite leurs types d'ARN.

In [ ]:
import subprocess

index='/content/drive/MyDrive/liste_sp_mappees.txt'
with open(index, 'r') as x:
    L = [line.strip() for line in x]
L2=[]
for nom_sp in L[:]:
  bash=f"curl -s https://rnacentral.org/api/v1/overlap/region/{nom_sp}/5:10000-200000 | sed -E 's/[{{.*,}}]/\\n/g' | egrep 'biotype' | cut -d ':' -f 2 | sort | uniq | sed -E 's/^\"(.*[^\"])\"$/\\1/g'"
  result = subprocess.run(bash, shell=True, capture_output=True, text=True)
  L2.append([nom_sp,result.stdout.strip()])

print(L2)

index2='/content/drive/MyDrive/liste_arn_et_sp.txt'

with open(index2, 'w') as liste:
    for item in L2:
        liste.write(f"{item}\n")

[['ailuropoda_melanoleuca', 'tRNA'], ['arabidopsis_lyrata', 'tRNA'], ['arabidopsis_thaliana', 'lncRNA\nmisc_RNA\nsiRNA\nsnoRNA\nsnRNA\nsRNA\ntRNA'], ['arabis_alpina', 'tRNA'], ['aspergillus_oryzae', 'rRNA\ntRNA'], ['betta_splendens', 'snRNA'], ['bos_grunniens', 'lncRNA'], ['bos_indicus_hybrid', 'lncRNA'], ['bos_taurus', 'piRNA\ntRNA'], ['brachypodium_distachyon', 'misc_RNA\nrRNA'], ['callithrix_jacchus', 'rRNA'], ['canis_lupus_familiaris', 'miRNA\npre_miRNA'], ['capsicum_annuum', 'lncRNA'], ['cervus_hanglu_yarkandensis', 'lncRNA'], ['chlamydomonas_reinhardtii', 'lncRNA\nmisc_RNA'], ['chlorocebus_sabaeus', 'Y_RNA'], ['ciona_intestinalis', 'miRNA\nmisc_RNA'], ['coturnix_japonica', 'lncRNA'], ['cryptococcus_neoformans', 'tRNA'], ['cucumis_sativus', 'tRNA'], ['cyanidioschyzon_merolae', 'misc_RNA\ntRNA'], ['denticeps_clupeoides', 'lncRNA'], ['dictyostelium_discoideum', 'tRNA'], ['echeneis_naucrates', 'snoRNA'], ['equus_asinus', 'lncRNA'], ['equus_caballus', 'lncRNA'], ['fusarium_solani', 'r

J'ai maintenant les données que je souhaite. Je vais pouvoir les analyser.



In [3]:
import matplotlib.pyplot as plt
import pandas as pd
import ast

with open('/content/drive/MyDrive/liste_arn_et_sp.txt', 'r') as file:
    lines = file.readlines()

# Convertir les lignes en listes en utilisant ast.literal_eval
data_list = [ast.literal_eval(line.strip()) for line in lines]
print(data_list)
# Créer un DataFrame
df = pd.DataFrame(data_list, columns=['espèce', 'RNAtype'])

df['RNAtype'] = df['RNAtype'].str.split('\n')  # Séparer les types par saut de ligne

# Exploser le DataFrame pour avoir une ligne par type d'ARN
df_exploded = df.explode('RNAtype')



index2='/content/drive/MyDrive/dataframe_chr510000-200000_typearn.txt'

# Afficher le DataFrame final
print(df_exploded)
df_exploded.to_excel("/content/drive/MyDrive/output.xlsx")
#Essayons de visualiser concrètement ces données avec des histogrammes de répartitions.

comptages_ARN=df_exploded['RNAtype'].value_counts()
#print(comptages_ARN)

#barchart avec le comptages
#plt.figure(figsize=(10, 6))
#comptages_ARN.plot(kind='bar', color='red')
#plt.ylabel('Nombre sp où RNA est présent')
#plt.xticks(rotation=45)
#plt.grid(axis='y')
#plt.tight_layout()
#plt.show()





[['ailuropoda_melanoleuca', 'tRNA'], ['arabidopsis_lyrata', 'tRNA'], ['arabidopsis_thaliana', 'lncRNA\nmisc_RNA\nsiRNA\nsnoRNA\nsnRNA\nsRNA\ntRNA'], ['arabis_alpina', 'tRNA'], ['aspergillus_oryzae', 'rRNA\ntRNA'], ['betta_splendens', 'snRNA'], ['bos_grunniens', 'lncRNA'], ['bos_indicus_hybrid', 'lncRNA'], ['bos_taurus', 'piRNA\ntRNA'], ['brachypodium_distachyon', 'misc_RNA\nrRNA'], ['callithrix_jacchus', 'rRNA'], ['canis_lupus_familiaris', 'miRNA\npre_miRNA'], ['capsicum_annuum', 'lncRNA'], ['cervus_hanglu_yarkandensis', 'lncRNA'], ['chlamydomonas_reinhardtii', 'lncRNA\nmisc_RNA'], ['chlorocebus_sabaeus', 'Y_RNA'], ['ciona_intestinalis', 'miRNA\nmisc_RNA'], ['coturnix_japonica', 'lncRNA'], ['cryptococcus_neoformans', 'tRNA'], ['cucumis_sativus', 'tRNA'], ['cyanidioschyzon_merolae', 'misc_RNA\ntRNA'], ['denticeps_clupeoides', 'lncRNA'], ['dictyostelium_discoideum', 'tRNA'], ['echeneis_naucrates', 'snoRNA'], ['equus_asinus', 'lncRNA'], ['equus_caballus', 'lncRNA'], ['fusarium_solani', 'r